In [7]:
import numpy as np
import pandas as pd
import os
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import random

In [8]:
# getting the text file
!wget "www.gutenberg.org/files/1342/1342-0.txt"

--2024-10-22 16:14:20--  http://www.gutenberg.org/files/1342/1342-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.gutenberg.org/files/1342/1342-0.txt [following]
--2024-10-22 16:14:20--  https://www.gutenberg.org/files/1342/1342-0.txt
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 752583 (735K) [text/plain]
Saving to: ‘1342-0.txt’

1342-0.txt          100%[===================>] 734.94K  1.87MB/s    in 0.4s    

2024-10-22 16:14:21 (1.87 MB/s) - ‘1342-0.txt’ saved [752583/752583]



In [9]:
import requests

# URL of the text file
url = "http://www.gutenberg.org/files/1342/1342-0.txt"

# Download the file
response = requests.get(url)
if response.status_code == 200:
    # Save the content to a local file
    with open("pride_and_prejudice.txt", "w", encoding="utf-8") as f:
        f.write(response.text)

# Path to the downloaded file
novel_path = "pride_and_prejudice.txt"

In [10]:
# Function to read the novel
def read_novel(novel_path):
    txt = []
    with open(novel_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line == '----------':
                break
            if line != '':
                txt.append(line)
    return txt

# Read the novel
novel_lines = read_novel(novel_path)
print("Number of lines =", len(novel_lines))

Number of lines = 11574


In [11]:
# Function to clean and count words
def clean_and_count_words(novel_path):
    word_count = 0
    cleaned_text = []
    with open(novel_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line == '----------':
                break
            if line != '':
                # Tokenize the line into words using regex
                words = re.findall(r'\b\w+\b', line)
                word_count += len(words)
                cleaned_text.extend(words)

    return cleaned_text  # Return the list of words, not the count

# Clean and count words
cleaned_text = clean_and_count_words(novel_path)
print("Number of words =", len(cleaned_text))

Number of words = 128769


In [12]:
def make_markov_model(cleaned_text, n_gram=3):
    markov_model = {}
    for i in range(len(cleaned_text) - n_gram):
        curr_state, next_state = "", ""
        for j in range(n_gram):
            curr_index = i + j
            if curr_index < len(cleaned_text):
                curr_state += cleaned_text[curr_index] + " "
            next_index = i + j + n_gram
            if next_index < len(cleaned_text):
                next_state += cleaned_text[next_index] + " "

        curr_state = curr_state.strip()
        next_state = next_state.strip()

        if curr_state not in markov_model:
            markov_model[curr_state] = {}
            markov_model[curr_state][next_state] = 1
        else:
            if next_state in markov_model[curr_state]:
                markov_model[curr_state][next_state] += 1
            else:
                markov_model[curr_state][next_state] = 1

    # calculating transition probabilities
    for curr_state, transition in markov_model.items():
        total = sum(transition.values())
        for state, count in transition.items():
            markov_model[curr_state][state] = count / total

    return markov_model

In [13]:
pp_markov_model = make_markov_model(cleaned_text)

In [14]:
print("number of states = ", len(pp_markov_model.keys()))

number of states =  109519


In [15]:
def generate_story(pp_markov_model, limit=100, start='love'):
    n = 0
    curr_state = start
    next_state = None
    story = ""
    story+=curr_state+" "
    while n<limit:
        next_state = random.choices(list(pp_markov_model[curr_state].keys()),
                                    list(pp_markov_model[curr_state].values()))

        curr_state = next_state[0]
        story+=curr_state+" "
        n+=1
    return story

In [16]:
# Print all states in the Markov model
all_states = list(pp_markov_model.keys())
print("All states in the Markov model: \n", all_states)

# Choose a valid starting state from the available states
valid_starting_state = random.choice(all_states)

# Generate a story with the chosen starting state
generated_story = generate_story(pp_markov_model, limit=100, start=valid_starting_state)
print("Generated Story: \n", generated_story)

All states in the Markov model: 
 ['START OF THE', 'OF THE PROJECT', 'THE PROJECT GUTENBERG', 'PROJECT GUTENBERG EBOOK', 'GUTENBERG EBOOK 1342', 'EBOOK 1342 Illustration', '1342 Illustration GEORGE', 'Illustration GEORGE ALLEN', 'GEORGE ALLEN PUBLISHER', 'ALLEN PUBLISHER 156', 'PUBLISHER 156 CHARING', '156 CHARING CROSS', 'CHARING CROSS ROAD', 'CROSS ROAD LONDON', 'ROAD LONDON RUSKIN', 'LONDON RUSKIN HOUSE', 'RUSKIN HOUSE Illustration', 'HOUSE Illustration _Reading', 'Illustration _Reading Jane', '_Reading Jane s', 'Jane s Letters', 's Letters _', 'Letters _ _Chap', '_ _Chap 34', '_Chap 34 _', '34 _ PRIDE', '_ PRIDE and', 'PRIDE and PREJUDICE', 'and PREJUDICE by', 'PREJUDICE by Jane', 'by Jane Austen', 'Jane Austen with', 'Austen with a', 'with a Preface', 'a Preface by', 'Preface by George', 'by George Saintsbury', 'George Saintsbury and', 'Saintsbury and Illustrations', 'and Illustrations by', 'Illustrations by Hugh', 'by Hugh Thomson', 'Hugh Thomson Illustration', 'Thomson Illustrat